# Import

In [ ]:
path_essentials = '/Users/Setup3-PC/Desktop/qudi/logic/setup3_logic/script_logic_essentials.py'
import os
import time
print('{}, working dir: {}'.format(time.ctime(), os.getcwd()))
#path_essentials = 'scripts/Simon/script_logic_essentials.py'
exec(open(path_essentials).read())

In [ ]:
memory_dict=OrderedDict()

# Common parameters

In [ ]:
# MW parameters
common_rabi_period = 82.85e-9
common_mw_freq = 1.47977e9
common_mw_amp = 0.25

# Cnot gates
common_mw_cnot_freq = common_mw_freq + 3.03e6
common_mw_cnot_period = 566.5e-9
common_mw_cnot_amp = 0.25/14.8
common_mw_cnot_freq2 = common_mw_freq
common_mw_cnot_period2 = 582.5e-9
common_mw_cnot_amp2 = 0.25/14.4

# N15 control
common_rf_period = 114e-6
common_rf_freq = 1.07457e6#1.9574e6
common_rf_amp = 0.25/2.5

common_refocus_time = 300

# Laser options

## Optimize position

In [ ]:
optimize_position()

## Laser On

In [ ]:
laser_on(pulser_on = False)

## Laser Off

In [ ]:
laser_off(pulser_on = False)

# Frequency Optimization parameters

In [ ]:
optimize_freq_dict=OrderedDict()
optimize_freq_dict['mw_optimization_method'] = 'pulsedodmr'
optimize_freq_dict['optimize_generate_new'] = False
optimize_freq_dict['save_tag'] = 'optimize_frequency'
optimize_freq_dict['optimize_fit_method'] = 'N15'
optimize_freq_dict['parameters2update'] = {'center': 'microwave_frequency'}

optimize_freq_dict['name'] = 'Pulsed_ODMR'
optimize_freq_dict['rabi_period'] = 1800e-9
optimize_freq_dict['freq_start'] = 4.127e9
optimize_freq_dict['freq_step'] = 0.2e6
optimize_freq_dict['num_of_points'] = 50
optimize_freq_dict['microwave_amplitude'] = common_mw_amp/15
optimize_freq_dict['meas_type'] = conventional_measurement
optimize_freq_dict['meas_info'] = add_conventional_information

optimize_freq_dict['measurement_time'] = 30
optimize_freq_dict['optimize_time'] = None
optimize_freq_dict['optimize_generate_new'] = True




In [ ]:
def setup_ramsey_params(tau):
    qmeas=OrderedDict()
    qmeas['name'] = 'Ramsey'
    qmeas['rabi_period'] = common_rabi_period
    qmeas['microwave_frequency'] = common_mw_freq+100e3
    qmeas['microwave_amplitude'] = common_mw_amp/1
    qmeas['tau_start'] = 10e-6
    qmeas['tau_step'] = 100e-9
    qmeas['num_of_points'] = 1
    qmeas['alternating'] = True
    
    qmeas['from_list'] = False
    qmeas['tau_list'] = np.linspace(100,100*50,50) * 1e-9
    
    qmeas['measurement_time'] = 1500
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    
    return qmeas

def run_ramsey(mes_params):
    do_experiment(experiment='ramsey', qm_dict=qmeas, meas_type=conventional_measurement, 
        meas_info=add_conventional_information, generate_new=True, save_tag='Ramsey',load_tag='')

def calc_new_tau():
    # todo: dummy
    import random
    return random.randint(100,1000)
    

for idx_epoch in range(0, 10):
    tau = calc_new_tau()
    qmeas = setup_ramsey_params()
    run_ramsey(qmeas)
    


# Fit from script


In [ ]:
import copy

pulsedmeasurementlogic.fc.set_current_fit('t2')

data = pulsedmeasurementlogic.signal_data

params={'order': {'value':16, 'vary': False}}
params['amp'] = {'value':40e3, 'min':10e3, 'max':75e3, 'vary': True}
#freq = np.round(1/2/187.2e-9)
#params['frequency'] = {'value':freq, 'min':2.66e6, 'max':2.666e6, 'vary': True}
#params['frequency'] = {'value':freq, 'vary': False}
params['phase'] = {'value':np.pi, 'vary': False}
#params['offset'] = {'value':0.87, 'vary': False}


x_fit, y_fit, result = pulsedmeasurementlogic.fc.do_fit(data[0], data[1], params)

fit_data = np.array([x_fit, y_fit])

pulsedmeasurementlogic.signal_fit_data = fit_data
pulsedmeasurementlogic.fit_result = copy.deepcopy(pulsedmeasurementlogic.fc.current_fit_result)
pulsedmeasurementlogic.sigFitUpdated.emit(pulsedmeasurementlogic.fc.current_fit, 
                                          pulsedmeasurementlogic.signal_fit_data, 
                                          pulsedmeasurementlogic.fit_result,
                                          False)

result.result_str_dict

# Further Measurements

## Rabi alternating

In [ ]:
qmeas=OrderedDict()
qmeas['name'] = 'Rabi'
qmeas['tau_start'] = 0e-9
qmeas['tau_step'] = 20e-9
qmeas['num_of_points'] = 50
qmeas['microwave_frequency'] = common_mw_freq #+ 3.03e6
qmeas['microwave_amplitude'] = 0.25/14.4

qmeas['measurement_time'] = None

qmeas['optimize_time'] = None
qmeas['freq_optimize_time'] = None
qmeas['fit_experiment'] = 'cosine'

do_experiment(experiment='rabi_alternating', qm_dict=qmeas, meas_type=conventional_measurement, meas_info=add_conventional_information,
              generate_new=True, save_tag='Rabi_'+str(round(qmeas['microwave_amplitude']*1000))+'mV',load_tag='')

In [ ]:
test = [0,2,3,4]
test1 = [5,7,8,9]

test2 = [0,3,6]

[x for x in test2 if x-1 in np.concatenate((test,test1))]


In [ ]:
bright_charge = singleshotlogic.bright_states
dark_charge = singleshotlogic.dark_states

bright_filtered = [x for x in bright_charge if x-1 in np.concatenate((bright_charge,dark_charge))]
dark_filtered = [x for x in dark_charge if x-1 in np.concatenate((bright_charge,dark_charge))]

bright = [x for x in bright_filtered if x%2!=0]
dark = [x for x in dark_filtered if x%2!=0]

bright_alt = [x for x in bright_filtered if x%2==0]
dark_alt = [x for x in dark_filtered if x%2==0]

mapped_state = len(bright)/(len(bright)+len(dark))
mapped_alt = len(bright_alt)/(len(bright_alt)+len(dark_alt))

#sorted(np.concatenate((bright_charge,dark_charge)))
#singleshotlogic.charge_state[:10]
print(mapped_alt)
print(mapped_state)

In [ ]:
np.where(singleshotlogic.charge_state < singleshotlogic.charge_threshold)[0]